# Loading JSON Files

In [1]:
from loader import read_json_file

vector_retriever_retrieved_nodes = read_json_file("input/vector.json").get("nodes")
hybrid_retriever_retrieved_nodes = read_json_file("input/hybrid.json").get("nodes")

# Getting Chunks

In [2]:
vector_retriever_retrieved_chunks = [node["text"] for node in vector_retriever_retrieved_nodes]
hybrid_retriever_retrieved_chunks = [node["text"] for node in hybrid_retriever_retrieved_nodes]

# Finding Overlapping and Non-Overlapping Chunks

In [3]:
from overlap import find_overlapping_chunks

chunks = find_overlapping_chunks(vector_retriever_retrieved_chunks, hybrid_retriever_retrieved_chunks, 0.4)

In [4]:
overlapping_chunks, vector_only_chunks, hybrid_only_chunks = chunks.get("overlapping_chunks"), chunks.get("retriever_1_only_chunks"), chunks.get("retriever_2_only_chunks")

In [5]:
len(overlapping_chunks), len(vector_only_chunks), len(hybrid_only_chunks)

(9, 11, 11)

This means there are 9 overlapping chunks and 11 non-overlapping chunks for each retriever

In [6]:
query = read_json_file("input/vector.json").get("original_question")
query

'Tell me about the early life of Hazrat Ali'

In [13]:
from openai import OpenAI
from config import Config


def evaluate_retrievers(query, overlapping_chunks, retriever_1_only_chunks, retriever_2_only_chunks):
    
    prompt = f"""Analyze the relevance of text chunks to the following query: "{query}"

Chunk Categories:
1. Overlapping Chunks (50%+ similarity): {overlapping_chunks}
2. Chunks from Retriever 1 Only: {retriever_1_only_chunks}
3. Chunks from Retriever 2 Only: {retriever_2_only_chunks}

For each category, please:
- Rate the relevance of each chunk on a scale of 1-10
- Provide a brief explanation for the relevance score
- Highlight any significant insights about the retriever's performance

Format your response as a JSON with the following structure:
{{
    "overlapping_chunks": [
        {{"chunk": "chunk text", "relevance_score": 7, "explanation": "..."}}
    ],
    "retriever_1_only_chunks": [
        {{"chunk": "chunk text", "relevance_score": 5, "explanation": "..."}}
    ],
    "retriever_2_only_chunks": [
        {{"chunk": "chunk text", "relevance_score": 8, "explanation": "..."}}
    ],
    "overall_assessment": "Summary of retriever performance"
}}"""
    
    client = OpenAI(api_key=Config.OPENAI_API_KEY)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a QA specialist evaluating the relevance of text chunks to a query."},
            {"role": "user", "content": "User: " + prompt}
        ]
    )

    return completion.choices[0].message

In [14]:
# from evaluator import evaluate_retrievers

response = evaluate_retrievers(query, overlapping_chunks, vector_only_chunks, hybrid_only_chunks)

In [15]:
response

ChatCompletionMessage(content='```json\n{\n    "overlapping_chunks": [\n        {\n            "chunk": "[248]Imam al-Hadi (a.s) visited his grandfather Amir’ul- Mu’minin\\nwith this wonderful ziyarah in which he mentioned the virtues of Imam Ali\\n(a.s) and the political and social problems he suffered at that age. We\\nmention here some passages from the ziyarah: ...",\n            "relevance_score": 4,\n            "explanation": "This chunk discusses virtues and situations faced by Imam Ali, but it primarily focuses on later events and does not specifically address his early life."\n        },\n        {\n            "chunk": "Many a time, when he wants to enter his room, I say to him, ‘You do not enter the room until you recite a surah.’...‘This young boy grew up in Medina between black walls. How did he obtain this immense knowledge? Glory be to Allah!’",\n            "relevance_score": 6,\n            "explanation": "This chunk touches on Ali\'s upbringing and knowledge, suggest

In [20]:
response.content.lstrip("```").lstrip("\n").rstrip("\n").rstrip("```").lstrip("json")

'\n{\n    "overlapping_chunks": [\n        {\n            "chunk": "[248]Imam al-Hadi (a.s) visited his grandfather Amir’ul- Mu’minin\\nwith this wonderful ziyarah in which he mentioned the virtues of Imam Ali\\n(a.s) and the political and social problems he suffered at that age. We\\nmention here some passages from the ziyarah: ...",\n            "relevance_score": 4,\n            "explanation": "This chunk discusses virtues and situations faced by Imam Ali, but it primarily focuses on later events and does not specifically address his early life."\n        },\n        {\n            "chunk": "Many a time, when he wants to enter his room, I say to him, ‘You do not enter the room until you recite a surah.’...‘This young boy grew up in Medina between black walls. How did he obtain this immense knowledge? Glory be to Allah!’",\n            "relevance_score": 6,\n            "explanation": "This chunk touches on Ali\'s upbringing and knowledge, suggesting a background that influences his 

In [21]:
import json

json.loads(response.content.lstrip("```").lstrip("\n").rstrip("\n").rstrip("```").lstrip("json"))

{'overlapping_chunks': [{'chunk': '[248]Imam al-Hadi (a.s) visited his grandfather Amir’ul- Mu’minin\nwith this wonderful ziyarah in which he mentioned the virtues of Imam Ali\n(a.s) and the political and social problems he suffered at that age. We\nmention here some passages from the ziyarah: ...',
   'relevance_score': 4,
   'explanation': 'This chunk discusses virtues and situations faced by Imam Ali, but it primarily focuses on later events and does not specifically address his early life.'},
  {'chunk': 'Many a time, when he wants to enter his room, I say to him, ‘You do not enter the room until you recite a surah.’...‘This young boy grew up in Medina between black walls. How did he obtain this immense knowledge? Glory be to Allah!’',
   'relevance_score': 6,
   'explanation': "This chunk touches on Ali's upbringing and knowledge, suggesting a background that influences his character, but lacks detail specifically about his early life."},
  {'chunk': 'When he feared a sedition fro

In [22]:
# store it in json file
with open("output/evaluation.json", "w") as f:
    json.dump(json.loads(response.content.lstrip("```").lstrip("\n").rstrip("\n").rstrip("```").lstrip("json")), f, indent=4)